# Intro 

Here are a few situations where data sets don't work well:

* The data change frequently. It doesn't really make sense to regenerate a data set of stock prices, for example, and download it every minute. This approach would require a lot of bandwidth, and be very slow.
* You only want a small piece of a much larger data set. Reddit comments are one example. What if you want to pull just your own comments from reddit? It doesn't make much sense to download the entire reddit database, then filter it for a few items.
* It involves repeated computation. For example, Spotify has an API that can tell you the genre of a piece of music. You could theoretically create your own classifier and use it to categorize music, but you'll never have as much data as Spotify does.

In cases like these, an application program interface (API) is the right solution. An **API is a set of methods and tools that allows different applications to interact with each other**. Programmers use APIs to query and retrieve data dynamically (which they can then integrate with their own apps). A client can retrieve information quickly and effectively through an API.

# API Requests
Organizations host their APIs on Web servers. When you type www.google.com in your browser's address bar, your computer is actually asking the www.google.com server for a Web page, which it then returns to your browser.

APIs work much the same way, except instead of your Web browser asking for a Web page, your program asks for data. The API usually returns this data in JavaScript Object Notation (JSON) format. We'll discuss JSON more later on in this mission.

We use the requests library http://www.python-requests.org/en/latest/ to send requests


## GET Requests
There are many different types of requests. The most common is a GET request, which we use to retrieve data. We'll explore the other types in later missions.

http://open-notify.org/ offers several API endpoints. An **endpoint is a server route for retrieving specific data** from an API. For example, the **/comments endpoint** on the reddit API might retrieve information about **comments**, while the /users endpoint might retrieve data about users.





In [2]:
import requests

# Make a get request to get the latest position of the ISS from the OpenNotify API.
response = requests.get("http://api.open-notify.org/iss-now.json")

status_code = response.status_code
print(status_code)

200


In [3]:
print(response.text)

{"timestamp": 1532348377, "iss_position": {"latitude": "-47.9044", "longitude": "-47.1942"}, "message": "success"}


In [8]:
print(response.json())

{'timestamp': 1532348377, 'iss_position': {'latitude': '-47.9044', 'longitude': '-47.1942'}, 'message': 'success'}


In [10]:
r = requests.get("http://api.open-notify.org/iss-pass.json")
status_code = r.status_code
print(status_code)

400


### Handling parameters

#### Example: ISS Position

You'll see that in the last example, we got a `400` status code, which indicates a bad request. If you look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

This endpoint returns the next time the ISS will pass over a given location on the Earth.

To request this information, we'll need to pass the coordinates for a specific location to the API. We do this by passing in two parameters, latitude and longitude.

To accomplish this, we can add an optional keyword argument, params, to our request. In this case, we need to pass in **two parameters**:

* lat - The latitude of the location
* lon - The longitude of the location
We can make a **dictionary that contains these parameters**, and then pass them into the function.

We can also do the same thing directly by adding the query parameters to the url, like this:
http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74

It's almost always **preferable to set up the parameters as a dictionary**, because the requests library we mentioned earlier takes care of certain issues, like properly formatting the query parameters.



In [11]:
parameters = {"lat": 40.71, "lon": -74}

# Make a get request with the parameters.
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Print the content of the response (the data the server returned)
print(response.content)


b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1532348846, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 605, \n      "risetime": 1532389397\n    }, \n    {\n      "duration": 628, \n      "risetime": 1532395168\n    }, \n    {\n      "duration": 559, \n      "risetime": 1532401035\n    }, \n    {\n      "duration": 565, \n      "risetime": 1532406885\n    }, \n    {\n      "duration": 635, \n      "risetime": 1532412678\n    }\n  ]\n}\n'


### JSON
JSON is the **primary format** for sending and receiving data through APIs. This format **encodes data structures like lists and dictionaries as strings** to ensure that machines can read them easily. 

Python offers **great support for JSON through its json library**. We can convert lists and dictionaries to JSON, and vice versa. Our ISS Pass data, for example, is a dictionary encoded as a string in JSON format.

The **JSON library** has two main methods:

* **dumps** -- Takes in a Python object, and converts it to a string
* **loads** -- Takes a JSON string, and converts it to a Python object

We can get the **content of a response as a Python object** by using the `.json()` method on the response.




In [12]:
json_data = response.json()
print(type(json_data))
print(json_data)


<class 'dict'>
{'message': 'success', 'request': {'altitude': 100, 'datetime': 1532348846, 'latitude': 40.71, 'longitude': -74.0, 'passes': 5}, 'response': [{'duration': 605, 'risetime': 1532389397}, {'duration': 628, 'risetime': 1532395168}, {'duration': 559, 'risetime': 1532401035}, {'duration': 565, 'risetime': 1532406885}, {'duration': 635, 'risetime': 1532412678}]}


In [19]:
# get the duration of the first pass of the ISS
print(json_data["response"][0]["duration"])

605


### Content type and headers

The server sends more than a status code and the data when it generates a response. It also sends **metadata containing information on how it generated the data and how to decode it**. This information **appears in the response headers**. We can access it using the .headers property that responses have.

The headers will appear as a dictionary. For now, the content-type within the headers is the most important key. It tells us the format of the response, and how to decode it. For the OpenNotify API, the format is JSON, which is why we could decode it with JSON earlier.

In [20]:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Mon, 23 Jul 2018 12:27:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '519', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


In [21]:
print(type(response.headers))

<class 'requests.structures.CaseInsensitiveDict'>


In [27]:
content_type = response.headers["Content-Type"]

print(content_type)

application/json
